# 2. Prepare Data

From https://machinelearningmastery.com/process-for-working-through-machine-learning-problems/

I preface data preparation with a data analysis phase that involves summarizing the attributes and visualizing them using scatter plots and histograms. I also like to describe in detail each attribute and relationships between attributes. This grunt work forces me to think about the data in the context of the problem before it is lost to the algorithms

The actual data preparation process is three step as follows:

* Step 1: Data Selection: Consider what data is available, what data is missing and what data can be removed.
* Step 2: Data Preprocessing: Organize your selected data by formatting, cleaning and sampling from it.
* Step 3: Data Transformation: Transform preprocessed data ready for machine learning by engineering features using scaling, attribute decomposition and attribute aggregation.


In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap

%matplotlib inline

In [3]:
# expands column width to maximum
pd.set_option('display.max_colwidth', -1)

In [4]:
from sklearn.linear_model import LinearRegression

## Retrieve resources
The data directory will be ignored by git and you will need to retrieve manually the resources:

extract the contents of this zip file into the data subdirectory: 
https://s3.amazonaws.com/drivendata/data/1/public/cd238763-ed29-4a46-8584-f9334d57ec94.zip
you should have "data/TrainingSet.csv" and "data/SubmissionRows.csv"


you will need as well to put this file in the data/ folder: 
https://gist.githubusercontent.com/pamelafox/986163/raw/f5f9db4f1b287804fd07ffb3296ed0036292bc7a/countryinfo.py

In [ ]:
df = pd.read_csv("data/TrainingSet.csv")

In [ ]:
# Looks at the first few lines to get an idea what the actual data looks like
df.head(3)

In [ ]:
# Last few lines
df.tail(3)

The first column looks like a row ID, let's double check that it's the case

In [ ]:
df.iloc[:,0].nunique() == df.shape[0]

There are as many rows in the dataframe as identifiers in the first column, let's reload the data using the first column as the index

In [ ]:
df = pd.read_csv('data/TrainingSet.csv', index_col=0)

In [ ]:
df.head(3)

In [ ]:
# What column names do we have
df.columns

Let's rename the columns to remove gaps and unnecessary info and to ease further coding when selecting sets of columns

In [ ]:
df.columns = [year[:4] for year in df.columns][:-3] + [col.replace(' ', '_') for col in df.columns.values[-3:]]

In [ ]:
# Get some basic stats about the training data
df.describe().T

In [ ]:
# Find unique countries
df.Country_Name.unique()

In [ ]:
print ("Number of distinct countries {}".format(df.Country_Name.nunique()))

In [ ]:
# Pandas can also tell us how many unique values are in each column
df.nunique()

In [ ]:
# Or the percentage of non-nul data in each column
(df.count() / df.shape[0] * 100)

## Submission data

In [ ]:
# read the data containing the rows we need to predict
df_submission = pd.read_csv('data/SubmissionRows.csv', index_col=0)

In [ ]:
df_submission.head(10)

In [ ]:
df_submission.shape

As we have the row ID as an index for both the training data (df) and the submission we can directly extract the data related to the submission index

In [ ]:
df_submission_in_data = df.loc[df_submission.index]

In [ ]:
df_submission_in_data.Country_Name.nunique()

Let's have a look at the Series Code in the sudmission data

In [ ]:
submission_codes = df_submission_in_data.Series_Code.unique()
len(submission_codes)

In [ ]:
df.Series_Code.isin(submission_codes).sum()

In [ ]:
# Or the percentage of non-nul data in each column
(df.count() / df.shape[0] * 100).iloc[:-4].plot(label='training')
(df_submission_in_data.count() / df_submission_in_data.shape[0] * 100).iloc[:-4].plot(label='submission')
plt.legend();

We have learned from the analysis of the submission data that:
1. Most countries are represented in the submission data, 206 out of 214
2. Only 7 codes are used and we have 1118 data points in the training data with these codes
3. The amount of missing data is fairly similar between the training and submission data

# What are we trying to achieve?

We have 737 indicators from 206 countries with data from 1972 to 2007.  
We would like to predict what these indicators will be in 2008 and 2012.  

A very simplistic way of predicting the future values of these indicators would be to do a simple linear regression for indicators with more than 1 data point in the last 35 years or use the only data point we have for indicators with a single value.

**Let's try to code this simplistic version**

In [ ]:
def make_prediction(row):
    data = row.loc['1972':'2007']
    nbr_data_points = data.count()
    if nbr_data_points < 2:
        pred_2008 = data.dropna().values
        pred_2012 = pred_2008
    
    else:
        years = data.dropna().index.values.astype(np.int).reshape(-1, 1)
        values = data.dropna().values
        
        #linear regression
        regr = LinearRegression()
        regr.fit(years, values)
        
        #predictions
        pred_2008 = regr.predict(np.array([2008]).reshape(-1, 1))
        pred_2012 = regr.predict(np.array([2012]).reshape(-1, 1))
        
    return pred_2008[0], pred_2012[0]

In [ ]:
df_simple_preds = pd.DataFrame(df_submission_in_data.apply(make_prediction, axis=1).tolist(), index=df_submission_in_data.index, columns=['2008','2012'])

In [ ]:
df_simple_preds.head()

Let's visualise the results

In [ ]:
def plot_predictions(df_train, df_pred, nbr_rows):
    rows_to_plot = np.random.choice(df_train.index.values, nbr_rows, replace=False)
    
    cmap = get_cmap('Set1')
    colors = cmap.colors
        
    fig, ax = plt.subplots(figsize=(12,10))
    for i,j in zip(rows_to_plot, range(nbr_rows)):
        if j >= len(colors): j -= len(colors)
        ax.plot(df_train.loc[i, '1972':'2007'].dropna().index.astype(int), 
                df_train.loc[i, '1972':'2007'].dropna().values, 
                label=df_train.loc[i, 'Country_Name']+ '/' + df_train.loc[i, 'Series_Name'],
                marker='o',
                linewidth=4,
                alpha=0.5,
                color=colors[j])
                
        ax.plot(df_pred.loc[i].index.astype(int), 
                df_pred.loc[i].values,
                marker='s',
                linewidth=4,
                markersize=10,
                color=colors[j])

    plt.legend(loc=2)

In [ ]:
plot_predictions(df_submission_in_data, df_simple_preds, 4)

These results are simplistic but not too bad for most indicators.  
We could improve easily the results by doing a polynomial regression or another type of more sophisticated regression.  

But what we are trying to achieve is find correlation between these indicators and all the other indicators present in the dataset to improve our predictions as we have a lot of missing data.  
And these predictions using correlations are where the difficulty of this exercise lies.

## Separating sub codes from the main code values

Let's first make all values upper case to make sure we compare correctly the codes

In [ ]:
df.Series_Code = df.Series_Code.str.upper()

In [ ]:
df.Series_Code.nunique()

We have 1305 different code names but most of them include similar codes which might be helpful when trying to find similarity within the data to help with our predictions.  
We will create a function that extract each of the subcodes and save them in separate column in function of their order in the code serie.

In [ ]:
# Exploring the code structure
df.Series_Code.apply(lambda x: len(x.split('.'))).plot(kind='hist');

In [ ]:
row_longest_code_serie = df.Series_Code.apply(lambda x: len(x.split('.'))).idxmax()
df.loc[row_longest_code_serie, 'Series_Code']

In [ ]:
len(df.loc[row_longest_code_serie, 'Series_Code'].split('.'))

We have a maximum of 7 separate elements in the code structure

In [ ]:
def explode_code(s):
    # split the code name using the '.'
    s_split = s.split('.')
    
    # save the first and last value of the serie
    last = s_split[-1]
    first = s_split[0]
    
    # reverse the order of the code series to keep the last code as first and first as last
    #s_split = s_split[::-1]
    
    # add NaN values until the list has 7 items in it
    s_split += [np.NaN] * (7 - len(s_split))
    
    # add the first and last value to the list
    s_split += [first, last]
    
    return s_split

In [ ]:
# We first create 9 new columns for each of the output of the function explode_code
# fill these columns with the ouput of the explode_code function by converting its ouput to a list and then to a dataframe using the orginal index of the dataframe

df[['Series_Code_'+ str(i) for i in range(7)] + ['Series_Code_First', 'Series_Code_Last']] =\
    pd.DataFrame(df.Series_Code.apply(explode_code).tolist(), index=df.index)

In [ ]:
df.iloc[:5,-11:]

In [ ]:
df.iloc[-5:,-11:]

## Adding the continent related to each country

In [ ]:
from data import countryinfo

In [ ]:
df_continent = pd.DataFrame(countryinfo.countries)

In [ ]:
df_continent.columns

We copy the continent information from the dataframe using the country names as index

In [ ]:
# we use the map function of pandas for looking the the country name in the index of the second df and outputing the continent information
df['Continent'] = df.Country_Name.map(df_continent.set_index('name').continent)

In [ ]:
df[['Continent','Country_Name']].iloc[::15000]

We have some countries with names that are different from our list of country/continent

In [ ]:
df[df.Continent.isna()].Country_Name.unique()

In [ ]:
df[df.Country_Name.str.contains('Bahamas', case=False)].Country_Name.unique()

In [ ]:
df_continent[df_continent.name.str.contains('Bahamas', case=False)]

### Continent missing values

Let's do some fuzzy matching to find the most likely candidate for the country listed in our database

In [ ]:
!pip3 install fuzzywuzzy

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
country_bad_name = df[df.Continent.isna()].Country_Name.unique()
countries = df_continent.name.values

In [ ]:
country = country_bad_name[0]
print(country)
process.extractOne(country, countries)

In [ ]:
df_fuzzy = pd.DataFrame([[country]+list(process.extractOne(country, countries)) for country in country_bad_name], 
                        columns=['training_name', 'replacement_name', 'score'])

In [ ]:
df_fuzzy.head()

In [ ]:
# Let's add the continent to this dataframe
df_fuzzy['Continent'] = df_fuzzy.replacement_name.map(df_continent.set_index('name').continent)

In [ ]:
df_fuzzy.head()

In [ ]:
# Replacement_name that we are most sure about
df_fuzzy[df_fuzzy.score > 86]

In [ ]:
# Replacement_name that we are most sure about
df_fuzzy[df_fuzzy.score < 86]

We are missing a few easy fix like Congo and Korea

Let's fill the missing continents using the above rule

In [ ]:
countries_to_correct = df.Country_Name.isin(df_fuzzy[df_fuzzy.score > 86].training_name)

df.loc[countries_to_correct, 'Continent'] =\
    df.loc[countries_to_correct, 'Country_Name'].map(
        df_fuzzy[df_fuzzy.score > 86].set_index('training_name').Continent)

#### Let's redo the above fuzzy matching but trying a different metric to catch more missing values

In [ ]:
country_bad_name = df[df.Continent.isna()].Country_Name.unique()
countries = df_continent.name.values

In [ ]:
country = country_bad_name[5]
print(country)
print(process.extractOne(country, countries))
print(process.extractOne(country, countries, scorer=fuzz.ratio))
print(process.extractOne(country, countries, scorer=fuzz.partial_ratio))
print(process.extractOne(country, countries, scorer=fuzz.token_sort_ratio))
print(process.extractOne(country, countries, scorer=fuzz.token_set_ratio))
print(process.extractOne(country, countries, scorer=fuzz.partial_token_set_ratio))

In [ ]:
df_fuzzy = pd.DataFrame([[country]+list(process.extractOne(country, 
                                                           countries, 
                                                           scorer=fuzz.ratio
                                                          )) for country in country_bad_name], 
                        columns=['training_name', 'replacement_name', 'score'])

In [ ]:
# Let's add the continent to this dataframe
df_fuzzy['Continent'] = df_fuzzy.replacement_name.map(df_continent.set_index('name').continent)

In [ ]:
# Replacement_name that we are most sure about
df_fuzzy[df_fuzzy.score > 79]

Let's fill the missing continents using the above rule

In [ ]:
countries_to_correct = df.Country_Name.isin(df_fuzzy[df_fuzzy.score > 79].training_name)

df.loc[countries_to_correct, 'Continent'] =\
    df.loc[countries_to_correct, 'Country_Name'].map(
        df_fuzzy[df_fuzzy.score > 79].set_index('training_name').Continent)

#### Manual corrections for Korea, Congo and China

In [ ]:
df_continent[df_continent.name.str.contains('congo', case=False)]

In [ ]:
df_continent[df_continent.name.str.contains('korea', case=False)]

In [ ]:
df.loc[df.Country_Name.str.contains('congo', case=False), 'Continent'] = 'Africa'
df.loc[df.Country_Name.str.contains('korea', case=False), 'Continent'] = 'Asia'
df.loc[df.Country_Name.str.contains('china', case=False), 'Continent'] = 'Asia'

What is left in our missing continent?

In [ ]:
country_bad_name = df[df.Continent.isna()].Country_Name.unique()
country_bad_name

In [ ]:
len(country_bad_name)

In [ ]:
df.Country_Name.isin(country_bad_name).sum() / df.shape[0] * 100

In [ ]:
df_submission_in_data.Country_Name.isin(country_bad_name).sum() / df_submission_in_data.shape[0] * 100

We have less than 5% of the data with a missing continent in the training and submission data so we can probably leave it as it is and just replace it with Missing

In [ ]:
df.loc[df.Country_Name.isin(country_bad_name), 'Continent'] = 'Missing'

In [ ]:
df.head()

## Exploring the subcodes

In [ ]:
from collections import Counter

10 most common subcodes

In [ ]:
Counter(df.loc[:, 'Series_Code_0':'Series_Code_Last'].values.flat).most_common(10)

Most common codes for each subcode column ignoring the nan values

In [ ]:
def most_common_code(array, codes):
    array_notna = array[~pd.isna(array)]
    common_code = np.array(Counter(array_notna).most_common(codes))[:,0]
    common_code = np.append(common_code, [np.nan]*(codes-len(common_code)))
    return common_code

In [ ]:
pd.DataFrame(data= np.apply_along_axis(func1d=most_common_code, 
                                       arr=df.loc[:, 'Series_Code_0':'Series_Code_Last'].values,
                                       axis=0,
                                      codes=15), 
             columns=df.loc[:, 'Series_Code_0':'Series_Code_Last'].columns)